In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import models
import cv2
import librosa
import matplotlib.pyplot as plt

2023-07-03 16:17:33.185711: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 16:17:33.330955: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 16:17:35.448345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def route_to_numpy_array(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (196, 256))
    # divide by 255 to normalize the data
    resized_image = resized_image.astype('float32') / 255
    return resized_image

In [3]:
model = tf.keras.models.load_model('models/my_model.h5')

In [15]:
filename = 'file2.wav'
img_path = 'assets/inference_audios/'

In [25]:
df = pd.DataFrame(columns=["filename","image"])
df.at[0,"filename"] = filename

In [26]:
df

,filename,image
0,file2.wav,NaN


In [29]:
plt.figure(figsize=(3,5))
path = img_path + '/' + df["filename"][0]
print(path)
audio, sample_rate = librosa.load(path, sr=None, mono=False)
sgram = librosa.stft(audio, n_fft=1024, hop_length=None)
simple_sgram, _ = librosa.magphase(sgram)
sgram_mel = librosa.feature.melspectrogram(S=simple_sgram, sr=sample_rate, n_mels=64)
sgram_amp_log = librosa.amplitude_to_db(sgram_mel, ref=np.min)
filename = 'assets/inference_audios/' + df['filename'][0].split('.')[0] + '.jpg'
librosa.display.specshow(sgram_amp_log, sr=sample_rate/2, x_axis="time",y_axis="mel")
plt.axis('off')
plt.savefig(filename, bbox_inches='tight', pad_inches=0)
plt.close()

assets/inference_audios//file2.wav


In [31]:
df

,filename,image
0,file2.wav,NaN


In [50]:
img[2]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01176471,
       0.01960784, 0.07058824, 0.05882353, 0.08235294, 0.08627451,
       0.07843138, 0.03529412, 0.03921569, 0.01960784, 0.00784314,
       0.00392157, 0.00784314, 0.00392157, 0.00784314, 0.00784314,
       0.00784314, 0.00784314, 0.00392157, 0.01568628, 0.01568

In [53]:
df["image"].values

array([array([[0.        , 0.        , 0.        , ..., 0.00392157, 0.00392157,
               0.        ],
              [0.        , 0.        , 0.        , ..., 0.00392157, 0.00392157,
               0.00392157],
              [0.        , 0.        , 0.        , ..., 0.00392157, 0.00392157,
               0.00392157],
              ...,
              [0.        , 0.        , 0.        , ..., 0.21176471, 0.20784314,
               0.19607843],
              [0.        , 0.        , 0.        , ..., 0.22352941, 0.21960784,
               0.20392157],
              [0.        , 0.        , 0.        , ..., 0.22352941, 0.20784314,
               0.20392157]], dtype=float32)                                    ],
      dtype=object)

In [55]:
df.at[0,"image"] = route_to_numpy_array(filename)
h = df["image"].values
h: np.ndarray = np.stack(h)
print(len(h))
h = h.reshape((len(h), 196, 256, 1))
transformed = h
# prediction = model.predict(np.expand_dims(img, axis=0))

1


In [57]:
prediction = model.predict(transformed)
print(prediction)

1/1 [==============================] - 0s 436ms/step
[[1.8385150e-03 1.2489394e-03 7.6116592e-01 2.3354733e-01 1.3502812e-05
  2.3053777e-04 2.1987519e-06 1.0796514e-03 1.5736903e-05 8.5761928e-04]]


In [71]:
preds = prediction[0].tolist()
max = 0
index = 0
for i in range(len(preds)):
    if preds[i] > max:
        max = preds[i]
        index = i
        
print(i)

9
